In [6]:
class Value:
    """ 
    A wrapper for representing a scalar value as a node in the computational graph
    The scalar value, assigned operator, its gradient and attach children nodes 
    will be stored in the node 
    """

    def __init__(self, data, _children=(), _op='', label=''):
        # Callable variables
        self.label = label
        self.data  = data
        self.grad  = 0
        # Internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        # Define the addition operator
        other = other if isinstance(other, Value) else Value(other)
        # Return a new Value type with
        #   data = self.data + other.data
        #   _prev = (self, other)
        #   _op  = '+'
        return Value(self.data + other.data, (self, other), '+')

    def __mul__(self, other):
        # Define the multiply operator
        other = other if isinstance(other, Value) else Value(other)
        # Return a new Value type with
        #   data = self.data * other.data
        #   _prev = (self, other)
        #   _op  = '*'
        return Value(self.data * other.data, (self, other), '*')

    def __pow__(self, other):
        # Define the power operator
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        # Return a new Value type with
        #   data = self.data ** other
        #   _prev = (self, )
        #   _op  = '^'
        return Value(self.data**other, (self,), f'**{other}')

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def set_label(self, label_str):
        assert isinstance(label_str, str), "given label is not a string or char"
        self.label = label_str

    def __repr__(self):
        return f"Value(label={self.label}, data={self.data}, grad={self.grad})"

In [9]:
a = Value(3.0, label='a')
b = Value(10.0, label='b')
c = Value(-2.0, label='c')
print(a, b, c)
d = a * b + c - a ** 2 + b * c 
d.set_label('d')
print(d)

Value(label=a, data=3.0, grad=0) Value(label=b, data=10.0, grad=0) Value(label=c, data=-2.0, grad=0)
Value(label=d, data=-1.0, grad=0)
